Nessie Spark SQL Demo with NBA Dataset
============================
This demo showcases how to use Nessie Python API along with Spark3 from Iceberg

Initialize Pyspark
----------------------------------------------
To get started, we will first have to do a few setup steps that give us everything we need
to get started with Nessie. In case you're interested in the detailed setup steps for Spark, you can check out the [docs](https://projectnessie.org/tools/iceberg/spark/).

The Binder server has downloaded spark and some data for us as well as started a Nessie server in the background. All we have to do is start Spark.

The below cell starts a local Spark session with parameters needed to configure Nessie. Each config option is followed by a comment explaining its purpose.

In [ ]:
import os
import findspark
from pyspark.sql import *
from pyspark import SparkConf
import pynessie

findspark.init()
pynessie_version = pynessie.__version__

conf = SparkConf()
# we need iceberg libraries and the nessie sql extensions
conf.set(
    "spark.jars.packages",
    f"org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.projectnessie:nessie-spark-3.2-extensions:{pynessie_version}",
)
# ensure python <-> java interactions are w/ pyarrow
conf.set("spark.sql.execution.pyarrow.enabled", "true")
# create catalog dev_catalog as an iceberg catalog
conf.set("spark.sql.catalog.dev_catalog", "org.apache.iceberg.spark.SparkCatalog")
# tell the dev_catalog that its a Nessie catalog
conf.set("spark.sql.catalog.dev_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
# set the location for Nessie catalog to store data. Spark writes to this directory
conf.set("spark.sql.catalog.dev_catalog.warehouse", "file://" + os.getcwd() + "/spark_warehouse/iceberg")
# set the location of the nessie server. In this demo its running locally. There are many ways to run it (see https://projectnessie.org/try/)
conf.set("spark.sql.catalog.dev_catalog.uri", "http://localhost:19120/api/v1")
# default branch for Nessie catalog to work on
conf.set("spark.sql.catalog.dev_catalog.ref", "main")
# use no authorization. Options are NONE AWS BASIC and aws implies running Nessie on a lambda
conf.set("spark.sql.catalog.dev_catalog.auth_type", "NONE")
# enable the extensions for both Nessie and Iceberg
conf.set(
    "spark.sql.extensions",
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSpark32SessionExtensions",
)
# finally, start up the Spark server
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie#nessie-spark-3.2-extensions added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6cba98e4-6e15-458e-a366-568683d289f7;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/jovyan/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.13.1 in central
	found org.projectnessie#nessie-spark-3.2-extensions;0.30.0 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.2_2.12/0.13.1/iceberg-spark-runtime-3.2_2.12-0.13.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.13.1!iceberg-spark-runtime-3.2_2.12.jar (1331ms)
downloading https://repo1.maven.org/maven2/org/projectnessie/nessie-spark-3.2-extensions/0.30.0/nessie-spark-3.2-extensions-0.30.0.jar ...
	[SUCCESSFUL ] org.projectnessie#nessie-spark-3.2-extensions;0.30.0!nessie-spark-3.2-extensions.jar (70ms)
:: resolution report :: resolve 13309ms :: artifacts dl 1405ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.13.1 from central in [default]
	org.projectnessie#nessie-spark-3.2-extensions;0.30.0 from central in [default]
	---------------------------------------------------------------------
	|                  |       

Spark Running


Solving Data Engineering problems with Nessie
============================

In this Demo we are a data engineer working at a fictional sports analytics blog. In order for the authors to write articles they have to have access to the relevant data. They need to be able to retrieve data quickly and be able to create charts with it.

We have been asked to collect and expose some information about basketball players. We have located some data sources and are now ready to start ingesting data into our data lakehouse. We will perform the ingestion steps on a Nessie branch to test and validate the data before exposing to the analysts.

Set up Nessie branches
----------------------------
Once all dependencies are configured, we can get started with ingesting our basketball data into `Nessie` with the following steps:

- Create a new branch named `dev`
- List all branches

It is worth mentioning that we don't have to explicitly create a `main` branch, since it's the default branch.

In [ ]:
spark.sql("CREATE BRANCH dev IN dev_catalog FROM main").toPandas()

,refType,name,hash
0,Branch,dev,2e1cfa82b035c26cbbbdae632cea070514eb8b773f616a...


We have created the branch `dev` and we can see the branch with the Nessie `hash` its currently pointing to.

Below we list all branches. Note that the auto created `main` branch already exists and both branches point at the same empty `hash` initially

In [ ]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,dev,2e1cfa82b035c26cbbbdae632cea070514eb8b773f616a...
1,Branch,main,2e1cfa82b035c26cbbbdae632cea070514eb8b773f616a...


Create tables under dev branch
-------------------------------------
Once we created the `dev` branch and verified that it exists, we can create some tables and add some data.

We create two tables under the `dev` branch:
- `salaries`
- `totals_stats`

These tables list the salaries per player per year and their stats per year.

To create the data we:

1. switch our branch context to dev
2. create the table
3. insert the data from an existing csv file. This csv file is already stored locally on the demo machine. A production use case would likely take feeds from official data sources

In [ ]:
spark.sql("USE REFERENCE dev IN dev_catalog")

# Creating `salaries` table
spark.sql(
    """CREATE TABLE IF NOT EXISTS dev_catalog.nba.salaries
            (Season STRING, Team STRING, Salary STRING, Player STRING) USING iceberg"""
)

spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW salaries_table USING csv
            OPTIONS (path "../datasets/nba/salaries.csv", header true)"""
)
spark.sql("INSERT INTO dev_catalog.nba.salaries SELECT * FROM salaries_table")

# Creating `totals_stats` table
spark.sql(
    """CREATE TABLE IF NOT EXISTS dev_catalog.nba.totals_stats (
             Season STRING, Age STRING, Team STRING, ORB STRING, DRB STRING, TRB STRING, AST STRING, STL STRING,
             BLK STRING, TOV STRING, PTS STRING, Player STRING, RSorPO STRING)
             USING iceberg"""
)
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW stats_table USING csv
            OPTIONS (path "../datasets/nba/totals_stats.csv", header true)"""
)
spark.sql("INSERT INTO dev_catalog.nba.totals_stats SELECT * FROM stats_table").toPandas()

""


Now we count the rows in our tables to ensure they are the same number as the csv files. Note we use the `table@branch` notation which overrides the context set by a `USE REFERENCE` command.

In [ ]:
table_count = spark.sql("select count(*) from dev_catalog.nba.`salaries@dev`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from salaries_table").toPandas().values[0][0]
assert table_count == csv_count
print(table_count)

table_count = spark.sql("select count(*) from dev_catalog.nba.`totals_stats@dev`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from stats_table").toPandas().values[0][0]
assert table_count == csv_count
print(table_count)

50
92


Check generated tables
----------------------------
Since we have been working solely on the `dev` branch, where we created 2 tables and added some data,
let's verify that the `main` branch was not altered by our changes.

In [ ]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary


And on the `dev` branch we expect to see two tables

In [ ]:
spark.sql("USE REFERENCE dev IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,totals_stats,False
1,nba,salaries,False


We can also verify that the `dev` and `main` branches point to different commits

In [ ]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,dev,70a8df769b477de5b9157691edef1efca8a640ae9f7137...
1,Branch,main,2e1cfa82b035c26cbbbdae632cea070514eb8b773f616a...


Dev promotion into main
-----------------------
Once we are done with our changes on the `dev` branch, we would like to merge those changes into `main`.
We merge `dev` into `main` via the Spark sql `merge` command.
Both branches should be at the same revision after merging/promotion.

In [ ]:
spark.sql("MERGE BRANCH dev INTO main IN dev_catalog").toPandas()

,name,hash
0,main,af5e8edd1b769f3840ee485c0c6fc6aeaaebeb10d6ad38...


We can verify that the `main` branch now contains the expected tables and row counts.

The tables are now on `main` and ready for consumption by our blog authors and analysts!

In [ ]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,main,af5e8edd1b769f3840ee485c0c6fc6aeaaebeb10d6ad38...
1,Branch,dev,70a8df769b477de5b9157691edef1efca8a640ae9f7137...


In [ ]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,salaries,False
1,nba,totals_stats,False


In [ ]:
table_count = spark.sql("select count(*) from dev_catalog.nba.salaries").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from salaries_table").toPandas().values[0][0]
assert table_count == csv_count
print(table_count)

table_count = spark.sql("select count(*) from dev_catalog.nba.totals_stats").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from stats_table").toPandas().values[0][0]
assert table_count == csv_count
print(table_count)

50
92


Perform regular ETL on the new tables
-------------------
Our analysts are happy with the data and we want to now regularly ingest data to keep things up to date. Our first ETL job consists of the following:

1. Update the salaries table to add new data
2. We have decided the `Age` column isn't required in the `totals_stats` table so we will drop the column
3. We create a new table to hold information about the players appearances in all star games

As always we will do this work on a branch and verify the results. This ETL job can then be set up to run nightly with new stats and salary information.

In [ ]:
spark.sql("CREATE BRANCH etl IN dev_catalog FROM main").toPandas()

,refType,name,hash
0,Branch,etl,af5e8edd1b769f3840ee485c0c6fc6aeaaebeb10d6ad38...


In [ ]:
# add some salaries for Kevin Durant
spark.sql("USE REFERENCE etl IN dev_catalog")
spark.sql(
    """INSERT INTO dev_catalog.nba.salaries VALUES
    ("2017-18", "Golden State Warriors", "$25000000", "Kevin Durant"),
    ("2018-19", "Golden State Warriors", "$30000000", "Kevin Durant"),
    ("2019-20", "Brooklyn Nets", "$37199000", "Kevin Durant"),
    ("2020-21", "Brooklyn Nets", "$39058950", "Kevin Durant")
    """
).toPandas()

""


In [ ]:
# Dropping a column in the `totals_stats` table
spark.sql("ALTER TABLE dev_catalog.nba.totals_stats DROP COLUMN Age").toPandas()

""


In [ ]:
# Creating `allstar_games_stats` table and viewing the contents
spark.sql(
    """CREATE TABLE IF NOT EXISTS dev_catalog.nba.allstar_games_stats (
          Season STRING, Age STRING, Team STRING, ORB STRING, TRB STRING, AST STRING, STL STRING, BLK STRING,
          TOV STRING, PF STRING, PTS STRING, Player STRING)
          USING iceberg"""
)
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW allstar_table USING csv
            OPTIONS (path "../datasets/nba/allstar_games_stats.csv", header true)"""
)
spark.sql("INSERT INTO dev_catalog.nba.allstar_games_stats SELECT * FROM allstar_table").toPandas()

# notice how we view the data on the etl branch via @etl
spark.sql("select count(*) from dev_catalog.nba.`allstar_games_stats@etl`").toPandas()

,count(1)
0,47


We can verify that the new table isn't on the `main` branch but is present on the etl branch

In [ ]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,salaries,False
1,nba,totals_stats,False


In [ ]:
spark.sql("USE REFERENCE etl IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,allstar_games_stats,False
1,nba,totals_stats,False
2,nba,salaries,False


Now that we are happy with the data we can again merge it into `main`

In [ ]:
spark.sql("MERGE BRANCH etl INTO main IN dev_catalog").toPandas()

,name,hash
0,main,e0f4167c8947d15161a8fedc376da020f1ecff63c5eea2...


Now lets verify that the changes exist on the `main` branch

In [ ]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,salaries,False
1,nba,allstar_games_stats,False
2,nba,totals_stats,False


In [ ]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,main,e0f4167c8947d15161a8fedc376da020f1ecff63c5eea2...
1,Branch,etl,957c1254ab0a3e3bd1e306669ebe7073e27a97966bcfda...
2,Branch,dev,70a8df769b477de5b9157691edef1efca8a640ae9f7137...


In [ ]:
table_count = spark.sql("select count(*) from dev_catalog.nba.allstar_games_stats").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from allstar_table").toPandas().values[0][0]
assert table_count == csv_count
print(table_count)

47


Create `experiment` branch
--------------------------------
As a data analyst we might want to carry out some experiments with some data, without affecting `main` in any way.
As in the previous examples, we can just get started by creating an `experiment` branch off of `main`
and carry out our experiment, which could consist of the following steps:
- drop `totals_stats` table
- add data to `salaries` table
- compare `experiment` and `main` tables

In [ ]:
spark.sql("CREATE BRANCH experiment IN dev_catalog FROM main").toPandas()
spark.sql("USE REFERENCE experiment IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,experiment,e0f4167c8947d15161a8fedc376da020f1ecff63c5eea2...


In [ ]:
# Drop the `totals_stats` table on the `experiment` branch
spark.sql("DROP TABLE dev_catalog.nba.totals_stats").toPandas()

""


In [ ]:
# add some salaries for Dirk Nowitzki
spark.sql(
    """INSERT INTO dev_catalog.nba.salaries VALUES
    ("2015-16", "Dallas Mavericks", "$8333333", "Dirk Nowitzki"),
    ("2016-17", "Dallas Mavericks", "$25000000", "Dirk Nowitzki"),
    ("2017-28", "Dallas Mavericks", "$5000000", "Dirk Nowitzki"),
    ("2018-19", "Dallas Mavericks", "$5000000", "Dirk Nowitzki")
    """
).toPandas()

""


In [ ]:
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,salaries,False
1,nba,allstar_games_stats,False


In [ ]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName,isTemporary
0,nba,salaries,False
1,nba,allstar_games_stats,False
2,nba,totals_stats,False


Let's take a look at the contents of the `salaries` table on the `experiment` branch.
Notice the use of the `nessie` catalog and the use of `@experiment` to view data on the `experiment` branch

In [ ]:
spark.sql("select count(*) from dev_catalog.nba.`salaries@experiment`").toPandas()

,count(1)
0,58


and compare to the contents of the `salaries` table on the `main` branch. Notice that we didn't have to specify `@branchName` as it defaulted
to the `main` branch

In [ ]:
spark.sql("select count(*) from dev_catalog.nba.salaries").toPandas()

,count(1)
0,54


And finally lets clean up after ourselves

In [ ]:
spark.sql("DROP BRANCH dev IN dev_catalog")
spark.sql("DROP BRANCH etl IN dev_catalog")
spark.sql("DROP BRANCH experiment IN dev_catalog")

DataFrame[status: string]